In [1]:
from pyspark import SparkConf,SparkContext

In [2]:
conf = SparkConf().setMaster("local[*]").setAppName("spark_try")
sc=SparkContext(conf=conf)

23/12/27 23:42:53 WARN Utils: Your hostname, hewens-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.108 instead (on interface en0)
23/12/27 23:42:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/12/27 23:42:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/27 23:42:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9],3)
rdd.glom().collect()

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

In [8]:
data = [12,3,4,5,6,7,8]
new_rdd = sc.parallelize(data)
new_rdd.getNumPartitions()

8

In [9]:
file_rdd = sc.textFile('./words.txt')
file_rdd.getNumPartitions()

2

In [11]:
file_rdd_flat = file_rdd.flatMap(lambda x:x.split())
file_rdd_flat.collect()

['spark',
 'spark',
 'sql',
 'sql',
 'sql',
 'python',
 'python',
 'github',
 'java',
 'java',
 'cpp',
 'cpp']

In [14]:
words_count = file_rdd_flat.map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)
words_count.collect()

[('python', 2),
 ('java', 2),
 ('cpp', 2),
 ('spark', 2),
 ('sql', 3),
 ('github', 1)]